In [ ]:
from diffusers import DiffusionPipeline
import torch
import numpy as np
import random
from PIL import Image
from diffusers.utils import make_image_grid

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    cache_dir='/mnt/A-NVME/jordankp/sdxl_models/')
base.to("cuda:1")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    cache_dir='/mnt/A-NVME/jordankp/sdxl_models/')
refiner.to("cuda:1")

n_steps = 50
high_noise_frac = 0.99
guidance_scale = 12

human_prompt_eng = 'a male or female gender diverse firefighter, focused on the face'
img_list = []
for seed in range(30):
    torch.manual_seed(seed)
    image = base(
        prompt=human_prompt_eng,
        negative_prompt=None,
        num_inference_steps=n_steps,
        denoising_end=high_noise_frac,
        guidance_scale=guidance_scale,
        output_type="latent"
    ).images
    image_human_prompt_eng = refiner(
        prompt=human_prompt_eng,
        negative_prompt=None,
        num_inference_steps=n_steps,
        denoising_start=high_noise_frac,
        image=image,
        guidance_scale=guidance_scale,
    ).images[0]

    img_list.append(image_human_prompt_eng)
    
make_image_grid(img_list, rows=6, cols=5)